In [2]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [3]:
import os
from dotenv import load_dotenv
import pandas as pd
import math
import requests
import json

In [4]:
load_dotenv()

places_key = os.getenv("PLACES_KEYAPI")
print(f"We have a Here token: {places_key[:3]}")

We have a Here token: AIz


In [5]:
companies = list(db["companies"].find({},{"name":1,"offices":1,"category_code":1}))

In [393]:
df = pd.DataFrame(companies)
display(df)

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"[{'description': '', 'address1': '710 - 2nd Av..."
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"[{'description': '', 'address1': 'Suite 200', ..."
3,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."
4,52cdef7c4bab8bd675297d8d,Digg,news,"[{'description': None, 'address1': '135 Missis..."
...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f5,Oriact,software,"[{'description': '', 'address1': 'LÃ¤rchenweg ..."
18797,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,"[{'description': '', 'address1': '', 'address2..."
18798,52cdef7f4bab8bd67529c6f8,goBookmaker,web,[]
18799,52cdef7f4bab8bd67529c6f7,AfterLogic,software,"[{'description': 'Livingston', 'address1': 'P...."


In [394]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

In [395]:
cleanData

,_id,name,category_code,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"{'description': 'Headquarters', 'address1': '4...",Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"{'description': '', 'address1': 'Suite 200', '...",,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,52cdef7c4bab8bd675297d90,Postini,web,"{'description': None, 'address1': '959 Skyway ...",None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f5,Oriact,software,"{'description': '', 'address1': 'LÃ¤rchenweg 7...",,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,47.088219,8.437163
18797,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,"{'description': '', 'address1': '', 'address2'...",,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18799,52cdef7f4bab8bd67529c6f7,AfterLogic,software,"{'description': 'Livingston', 'address1': 'P.O...",Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [396]:
cleanData = cleanData.drop(columns=["_id","offices"])

In [397]:
cleanData

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...,...
18796,Oriact,software,,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,47.088219,8.437163
18797,EnteGreat Solutions,software,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,goBookmaker,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18799,AfterLogic,software,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [17]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None

In [399]:
cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData[["latitude","longitude","location"]]

,latitude,longitude,location
0,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,NaN,NaN,None
3,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
...,...,...,...
18796,47.088219,8.437163,"{'type': 'Point', 'coordinates': [8.4371634, 4..."
18797,33.518885,-86.816068,"{'type': 'Point', 'coordinates': [-86.816068, ..."
18798,NaN,NaN,None
18799,40.793024,-74.323554,"{'type': 'Point', 'coordinates': [-74.3235539,..."


In [400]:
cleanData = cleanData.drop(columns=["latitude","longitude"])

In [401]:
cleanData.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,None
3,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."


In [402]:
cleanData

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,None
3,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."
...,...,...,...,...,...,...,...,...,...,...
18796,Oriact,software,,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,"{'type': 'Point', 'coordinates': [8.4371634, 4..."
18797,EnteGreat Solutions,software,,,,,Birmingham,AL,USA,"{'type': 'Point', 'coordinates': [-86.816068, ..."
18798,goBookmaker,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
18799,AfterLogic,software,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,"{'type': 'Point', 'coordinates': [-74.3235539,..."


In [403]:
cleanData.to_json("output/cleaned_offices.json", orient="records")

In [6]:
starbucks = pd.read_csv('./input/starbucks.csv')

In [7]:
starbucks = starbucks.drop(columns=['Store Number','Store Name','Ownership Type','Phone Number','Timezone'])

In [8]:
starbucks

,Brand,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude
0,Starbucks,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,1.53,42.51
1,Starbucks,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,55.47,25.42
2,Starbucks,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,55.47,25.39
3,Starbucks,Al Salam Street,Abu Dhabi,AZ,AE,NaN,54.38,24.48
4,Starbucks,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,54.54,24.51
...,...,...,...,...,...,...,...,...
25595,Starbucks,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,106.70,10.78
25596,Starbucks,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,106.71,10.72
25597,Starbucks,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,28.04,-26.15
25598,Starbucks,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,28.28,-25.79


In [9]:
starbucks['category_code'] = 'Coffe shop'

In [10]:
starbucks['description'] = 'Coffe shop'

In [11]:
starbucks['address2'] = None

In [12]:
starbucks

,Brand,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude,category_code,description,address2
0,Starbucks,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,1.53,42.51,Coffe shop,Coffe shop,None
1,Starbucks,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,55.47,25.42,Coffe shop,Coffe shop,None
2,Starbucks,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,55.47,25.39,Coffe shop,Coffe shop,None
3,Starbucks,Al Salam Street,Abu Dhabi,AZ,AE,NaN,54.38,24.48,Coffe shop,Coffe shop,None
4,Starbucks,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,54.54,24.51,Coffe shop,Coffe shop,None
...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,106.70,10.78,Coffe shop,Coffe shop,None
25596,Starbucks,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,106.71,10.72,Coffe shop,Coffe shop,None
25597,Starbucks,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,28.04,-26.15,Coffe shop,Coffe shop,None
25598,Starbucks,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,28.28,-25.79,Coffe shop,Coffe shop,None


In [18]:
starbucks["location"] = starbucks[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)

In [19]:
starbucks

,Brand,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude,category_code,description,address2,location
0,Starbucks,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,1.53,42.51,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [1.53, 42.51]}"
1,Starbucks,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,55.47,25.42,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [55.47, 25.42]}"
2,Starbucks,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,55.47,25.39,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [55.47, 25.39]}"
3,Starbucks,Al Salam Street,Abu Dhabi,AZ,AE,NaN,54.38,24.48,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [54.38, 24.48]}"
4,Starbucks,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,54.54,24.51,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [54.54, 24.51]}"
...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,106.70,10.78,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [106.7, 10.78]}"
25596,Starbucks,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,106.71,10.72,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [106.71, 10.72]}"
25597,Starbucks,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,28.04,-26.15,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [28.04, -26.15]}"
25598,Starbucks,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,28.28,-25.79,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [28.28, -25.79]}"


In [20]:
starbucks.columns = ['name','address1','city','state_code','country_code','zip_code','longitude','latitude','category_code','description','address2','location']

In [21]:
starbucks

,name,address1,city,state_code,country_code,zip_code,longitude,latitude,category_code,description,address2,location
0,Starbucks,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,1.53,42.51,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [1.53, 42.51]}"
1,Starbucks,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,55.47,25.42,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [55.47, 25.42]}"
2,Starbucks,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,55.47,25.39,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [55.47, 25.39]}"
3,Starbucks,Al Salam Street,Abu Dhabi,AZ,AE,NaN,54.38,24.48,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [54.38, 24.48]}"
4,Starbucks,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,54.54,24.51,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [54.54, 24.51]}"
...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,106.70,10.78,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [106.7, 10.78]}"
25596,Starbucks,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,106.71,10.72,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [106.71, 10.72]}"
25597,Starbucks,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,28.04,-26.15,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [28.04, -26.15]}"
25598,Starbucks,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,28.28,-25.79,Coffe shop,Coffe shop,None,"{'type': 'Point', 'coordinates': [28.28, -25.79]}"


In [22]:
starbucks = starbucks[['name', 'category_code', 'description', 'address1', 'address2','zip_code','city','state_code','country_code','location','longitude','latitude']]

In [23]:
#res_1 = starbucksCleaned.loc[starbucksCleaned.index()== 8028]
starbucksCleaned = starbucks.drop(index=8028)
starbucksCleaned

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location,longitude,latitude
0,Starbucks,Coffe shop,Coffe shop,"Av. Meritxell, 96",None,AD500,Andorra la Vella,7,AD,"{'type': 'Point', 'coordinates': [1.53, 42.51]}",1.53,42.51
1,Starbucks,Coffe shop,Coffe shop,"1 Street 69, Al Jarf",None,NaN,Ajman,AJ,AE,"{'type': 'Point', 'coordinates': [55.47, 25.42]}",55.47,25.42
2,Starbucks,Coffe shop,Coffe shop,Sheikh Khalifa Bin Zayed St.,None,NaN,Ajman,AJ,AE,"{'type': 'Point', 'coordinates': [55.47, 25.39]}",55.47,25.39
3,Starbucks,Coffe shop,Coffe shop,Al Salam Street,None,NaN,Abu Dhabi,AZ,AE,"{'type': 'Point', 'coordinates': [54.38, 24.48]}",54.38,24.48
4,Starbucks,Coffe shop,Coffe shop,"Khaldiya Area, Abu Dhabi Island",None,NaN,Abu Dhabi,AZ,AE,"{'type': 'Point', 'coordinates': [54.54, 24.51]}",54.54,24.51
...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,Coffe shop,Coffe shop,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",None,70000,Thành Phố Hồ Chí Minh,SG,VN,"{'type': 'Point', 'coordinates': [106.7, 10.78]}",106.70,10.78
25596,Starbucks,Coffe shop,Coffe shop,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",None,70000,Thành Phố Hồ Chí Minh,SG,VN,"{'type': 'Point', 'coordinates': [106.71, 10.72]}",106.71,10.72
25597,Starbucks,Coffe shop,Coffe shop,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",None,2194,Johannesburg,GT,ZA,"{'type': 'Point', 'coordinates': [28.04, -26.15]}",28.04,-26.15
25598,Starbucks,Coffe shop,Coffe shop,"Shop 61B, Central Square, Cnr Aramist & Coroba...",None,181,Menlyn,GT,ZA,"{'type': 'Point', 'coordinates': [28.28, -25.79]}",28.28,-25.79


In [24]:
starbucksCleaned.to_json("output/starbucksCleaned.json", orient="records")

In [25]:
airports = pd.read_csv('./input/airports.csv')

In [26]:
airports = airports.loc[airports['type']== 'large_airport']

In [27]:
airports = airports[['name','latitude_deg','longitude_deg','iso_country']]

In [28]:
airports

,name,latitude_deg,longitude_deg,iso_country
11825,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG
12289,Keflavik International Airport,63.985001,-22.605600,IS
12336,Priština International Airport,42.572800,21.035801,XK
12340,Duplicate - Kempegowda International Airport B...,0.100000,0.100000,IN
15348,Guodu air base,36.001741,117.632010,CN
...,...,...,...,...
55722,Chengdu Shuangliu International Airport,30.578501,103.946999,CN
55790,Ürümqi Diwopu International Airport,43.907101,87.474197,CN
55799,Taiping Airport,45.623402,126.250000,CN
55813,Zhoushuizi Airport,38.965698,121.539001,CN


In [29]:
airports["location"] = airports[["latitude_deg","longitude_deg"]].apply(lambda x:asGeoJSON(x.latitude_deg,x.longitude_deg), axis=1)
airports

,name,latitude_deg,longitude_deg,iso_country,location
11825,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,"{'type': 'Point', 'coordinates': [147.22000122..."
12289,Keflavik International Airport,63.985001,-22.605600,IS,"{'type': 'Point', 'coordinates': [-22.60560035..."
12336,Priština International Airport,42.572800,21.035801,XK,"{'type': 'Point', 'coordinates': [21.035801, 4..."
12340,Duplicate - Kempegowda International Airport B...,0.100000,0.100000,IN,"{'type': 'Point', 'coordinates': [0.1, 0.1]}"
15348,Guodu air base,36.001741,117.632010,CN,"{'type': 'Point', 'coordinates': [117.63201000..."
...,...,...,...,...,...
55722,Chengdu Shuangliu International Airport,30.578501,103.946999,CN,"{'type': 'Point', 'coordinates': [103.94699859..."
55790,Ürümqi Diwopu International Airport,43.907101,87.474197,CN,"{'type': 'Point', 'coordinates': [87.474197387..."
55799,Taiping Airport,45.623402,126.250000,CN,"{'type': 'Point', 'coordinates': [126.25, 45.6..."
55813,Zhoushuizi Airport,38.965698,121.539001,CN,"{'type': 'Point', 'coordinates': [121.53900146..."


In [30]:
airports

,name,latitude_deg,longitude_deg,iso_country,location
11825,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,"{'type': 'Point', 'coordinates': [147.22000122..."
12289,Keflavik International Airport,63.985001,-22.605600,IS,"{'type': 'Point', 'coordinates': [-22.60560035..."
12336,Priština International Airport,42.572800,21.035801,XK,"{'type': 'Point', 'coordinates': [21.035801, 4..."
12340,Duplicate - Kempegowda International Airport B...,0.100000,0.100000,IN,"{'type': 'Point', 'coordinates': [0.1, 0.1]}"
15348,Guodu air base,36.001741,117.632010,CN,"{'type': 'Point', 'coordinates': [117.63201000..."
...,...,...,...,...,...
55722,Chengdu Shuangliu International Airport,30.578501,103.946999,CN,"{'type': 'Point', 'coordinates': [103.94699859..."
55790,Ürümqi Diwopu International Airport,43.907101,87.474197,CN,"{'type': 'Point', 'coordinates': [87.474197387..."
55799,Taiping Airport,45.623402,126.250000,CN,"{'type': 'Point', 'coordinates': [126.25, 45.6..."
55813,Zhoushuizi Airport,38.965698,121.539001,CN,"{'type': 'Point', 'coordinates': [121.53900146..."


In [31]:
airports.columns = ['name','latitude','longitude','country','location']
airports

,name,latitude,longitude,country,location
11825,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,"{'type': 'Point', 'coordinates': [147.22000122..."
12289,Keflavik International Airport,63.985001,-22.605600,IS,"{'type': 'Point', 'coordinates': [-22.60560035..."
12336,Priština International Airport,42.572800,21.035801,XK,"{'type': 'Point', 'coordinates': [21.035801, 4..."
12340,Duplicate - Kempegowda International Airport B...,0.100000,0.100000,IN,"{'type': 'Point', 'coordinates': [0.1, 0.1]}"
15348,Guodu air base,36.001741,117.632010,CN,"{'type': 'Point', 'coordinates': [117.63201000..."
...,...,...,...,...,...
55722,Chengdu Shuangliu International Airport,30.578501,103.946999,CN,"{'type': 'Point', 'coordinates': [103.94699859..."
55790,Ürümqi Diwopu International Airport,43.907101,87.474197,CN,"{'type': 'Point', 'coordinates': [87.474197387..."
55799,Taiping Airport,45.623402,126.250000,CN,"{'type': 'Point', 'coordinates': [126.25, 45.6..."
55813,Zhoushuizi Airport,38.965698,121.539001,CN,"{'type': 'Point', 'coordinates': [121.53900146..."


In [32]:
airportsCleaned = airports

In [33]:
airportsCleaned.to_json("output/airportsCleaned.json", orient="records")

In [877]:
def requestPlaces(lat,long):
    token = os.getenv("PLACES_KEYAPI")
    baseUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    api_key = f'&key={token}'
    location = f'location={lat},{long}'
    radius = '&radius=5000'
    typ = '&type=primary_school'
    url = baseUrl+location+radius+typ+api_key
 
    res = requests.get(url)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

In [747]:
lst_airports = []
for i in airports.iterrows():
    lst_airports.append((i[1]['latitude'],i[1]['longitude']))

In [879]:
lst_schools =[]
for i in lst_airports:
    lst_schools.append(requestPlaces(i[0],i[1]))

In [35]:
dfSchools_1 = pd.DataFrame(lst_schools)

NameError: name 'lst_schools' is not defined

In [1015]:
dfSchools_1.drop(columns=['html_attributions','status','next_page_token'])

,results
0,"[{'geometry': {'location': {'lat': -9.477105, ..."
1,"[{'geometry': {'location': {'lat': 63.9879383,..."
2,"[{'geometry': {'location': {'lat': 42.5392836,..."
3,[]
4,"[{'geometry': {'location': {'lat': 35.984918, ..."
...,...
613,"[{'geometry': {'location': {'lat': 30.592556, ..."
614,"[{'geometry': {'location': {'lat': 43.914201, ..."
615,"[{'geometry': {'location': {'lat': 45.638272, ..."
616,[{'geometry': {'location': {'lat': 38.92901610...


In [1016]:
dfSchools_1 = dfSchools_1.explode('results')
dfSchools_1

,html_attributions,results,status,next_page_token
0,[],"{'geometry': {'location': {'lat': -9.477105, '...",OK,NaN
0,[],{'geometry': {'location': {'lat': -9.454445799...,OK,NaN
0,[],{'geometry': {'location': {'lat': -9.463550000...,OK,NaN
0,[],{'geometry': {'location': {'lat': -9.442291100...,OK,NaN
0,[],{'geometry': {'location': {'lat': -9.474749099...,OK,NaN
...,...,...,...,...
616,[],{'geometry': {'location': {'lat': 38.951720999...,OK,CrQCJQEAALq2_I0X3gHCF1J5mhyzK8o5s5djNtfpOAZCtf...
617,[],"{'geometry': {'location': {'lat': 41.648409, '...",OK,NaN
617,[],{'geometry': {'location': {'lat': 41.658028000...,OK,NaN
617,[],"{'geometry': {'location': {'lat': 41.653019, '...",OK,NaN


In [1017]:
dfSchools_2 = dfSchools_1[["results"]].apply(lambda r: r.results, result_type="expand", axis=1)
#cleanData = pd.concat([df,dfOfficeData], axis=1)

In [1018]:
dfSchools_2

,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,price_level
0,"{'location': {'lat': -9.477105, 'lng': 147.200...",https://maps.gstatic.com/mapfiles/place_api/ic...,d90a0e8f09af8b7c354cdde90b2de28d1fdb81a9,Bava International School & Kindergarten,{'open_now': True},"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJG4xAqrc2AmkRgyP5MQneuNE,"{'compound_code': 'G6F2+54 Port Moresby, Papua...",5.0,ChIJG4xAqrc2AmkRgyP5MQneuNE,GOOGLE,"[primary_school, school, point_of_interest, es...",2.0,"Cnr Taurama Rd &, Turua Avenue, Port Moresby",NaN
0,"{'location': {'lat': -9.454445799999998, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,7d436a41d981a6f8759342f8e75c401939414b64,Holy Rosary Primary School,{'open_now': False},NaN,ChIJcXOdVeY3AmkRjbm_nzgNghw,"{'compound_code': 'G6W4+6V Port Moresby, Papua...",NaN,ChIJcXOdVeY3AmkRjbm_nzgNghw,GOOGLE,"[primary_school, school, point_of_interest, es...",NaN,"Saraga, Port Moresby",NaN
0,"{'location': {'lat': -9.463550000000001, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,8d9938513f4f74680f0ee795a653db59f9cb226a,Coronation Primary School,NaN,NaN,ChIJF2SWvtE2AmkRr8PdLpfT8Fo,"{'compound_code': 'G5PX+HG Port Moresby, Papua...",2.3,ChIJF2SWvtE2AmkRr8PdLpfT8Fo,GOOGLE,"[primary_school, school, point_of_interest, es...",7.0,"Kibi Pl., Port Moresby",NaN
0,"{'location': {'lat': -9.442291100000002, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,f0a143ac4ef78dbc051f91dc3bfa49a4edad268e,Wardstrip Demonstration Primary School,NaN,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJz5b7_cwwAmkRKDHQrjwn-JY,"{'compound_code': 'H55Q+3J Port Moresby, Papua...",4.0,ChIJz5b7_cwwAmkRKDHQrjwn-JY,GOOGLE,"[primary_school, school, point_of_interest, es...",2.0,"Godwin Street, Port Moresby",NaN
0,"{'location': {'lat': -9.474749099999999, 'lng'...",https://maps.gstatic.com/mapfiles/place_api/ic...,387ce9568bda461b163cd2b1caeb6190075ffde2,Bavaroko Primary School,NaN,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJ86_ehMc2AmkRzrsLtF9eowU,"{'compound_code': 'G6G4+43 Port Moresby, Papua...",4.0,ChIJ86_ehMc2AmkRzrsLtF9eowU,GOOGLE,"[primary_school, school, point_of_interest, es...",2.0,"Bava Street, Port Moresby",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,"{'location': {'lat': 38.95172099999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,6f1fbb9b01b93fbb2874bee9dab64447d4aa9407,Dalian Ganjingzi Experimental Primary School,NaN,NaN,ChIJYxYKEFlnjzUR6iQixUlM-YU,"{'compound_code': 'XG2C+MQ Ganjingzi, Dalian, ...",NaN,ChIJYxYKEFlnjzUR6iQixUlM-YU,GOOGLE,"[primary_school, school, point_of_interest, es...",NaN,"Hongyun Road, Ganjingzi, Dalian",NaN
617,"{'location': {'lat': 41.648409, 'lng': 123.476...",https://maps.gstatic.com/mapfiles/place_api/ic...,e03494a8b198fc99fc07a6299049c7ba38a70292,Taoxianzhen Central Elementary School,NaN,NaN,ChIJ6xVQjLKCL14Rw5R1d78rd5I,"{'compound_code': 'JFXG+9Q Hunnan, Shenyang, L...",NaN,ChIJ6xVQjLKCL14Rw5R1d78rd5I,GOOGLE,"[primary_school, school, point_of_interest, es...",NaN,"Hunnan, Shenyang",NaN
617,"{'location': {'lat': 41.65802800000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,d5e50a034a7035ffb83e19de8816eee6c3e06b6c,宁路小学,NaN,NaN,ChIJQZflrCGdL14RItFYZ6Y83yU,"{'compound_code': 'MG5F+6V Hunnan, Shenyang, L...",NaN,ChIJQZflrCGdL14RItFYZ6Y83yU,GOOGLE,"[primary_school, school, point_of_interest, es...",NaN,"Hunnan, Shenyang",NaN
617,"{'location': {'lat': 41.653019, 'lng': 123.532...",https://maps.gstatic.com/mapfiles/place_api/ic...,4cf3ba09d9476bf0a0f26e0be4e88a915dd8ed7f,Shenyang Conservatory of Music Affiliated Musi...,NaN,NaN,ChIJVaWlgRudL14RYgE3XEPfxIM,"{'compound_code': 'MG3M+63 Hunnan, Shenyang, L...",NaN,ChIJVaWlgRudL14RYgE3XEPfxIM,GOOGLE,"[primary_school, school, point_of_interest, es...",NaN,"304国道, Hunnan, Shenyang",NaN


In [1019]:
dfSchools_3 = dfSchools_2[['name','geometry']]
dfSchools_3

,name,geometry
0,Bava International School & Kindergarten,"{'location': {'lat': -9.477105, 'lng': 147.200..."
0,Holy Rosary Primary School,"{'location': {'lat': -9.454445799999998, 'lng'..."
0,Coronation Primary School,"{'location': {'lat': -9.463550000000001, 'lng'..."
0,Wardstrip Demonstration Primary School,"{'location': {'lat': -9.442291100000002, 'lng'..."
0,Bavaroko Primary School,"{'location': {'lat': -9.474749099999999, 'lng'..."
...,...,...
616,Dalian Ganjingzi Experimental Primary School,"{'location': {'lat': 38.95172099999999, 'lng':..."
617,Taoxianzhen Central Elementary School,"{'location': {'lat': 41.648409, 'lng': 123.476..."
617,宁路小学,"{'location': {'lat': 41.65802800000001, 'lng':..."
617,Shenyang Conservatory of Music Affiliated Musi...,"{'location': {'lat': 41.653019, 'lng': 123.532..."


In [1020]:
dfSchools_4 = dfSchools_3[["geometry"]].apply(lambda r: r.geometry, result_type="expand", axis=1)
dfSchools_4

,location,viewport
0,"{'lat': -9.477105, 'lng': 147.200274}","{'northeast': {'lat': -9.475704869708498, 'lng..."
0,"{'lat': -9.454445799999998, 'lng': 147.2072303}","{'northeast': {'lat': -9.452943819708498, 'lng..."
0,"{'lat': -9.463550000000001, 'lng': 147.1988583}","{'northeast': {'lat': -9.462426869708496, 'lng..."
0,"{'lat': -9.442291100000002, 'lng': 147.1891084}","{'northeast': {'lat': -9.440941319708498, 'lng..."
0,"{'lat': -9.474749099999999, 'lng': 147.205147}","{'northeast': {'lat': -9.473343619708498, 'lng..."
...,...,...
616,"{'lat': 38.95172099999999, 'lng': 121.521942}","{'northeast': {'lat': 38.9530007302915, 'lng':..."
617,"{'lat': 41.648409, 'lng': 123.4769939}","{'northeast': {'lat': 41.64984323029149, 'lng'..."
617,"{'lat': 41.65802800000001, 'lng': 123.524696}","{'northeast': {'lat': 41.6594177302915, 'lng':..."
617,"{'lat': 41.653019, 'lng': 123.532675}","{'northeast': {'lat': 41.6543776302915, 'lng':..."


In [1021]:
dfSchools_5 = dfSchools_4[["location"]].apply(lambda r: r.location, result_type="expand", axis=1)
dfSchools_5

,lat,lng
0,-9.477105,147.200274
0,-9.454446,147.207230
0,-9.463550,147.198858
0,-9.442291,147.189108
0,-9.474749,147.205147
...,...,...
616,38.951721,121.521942
617,41.648409,123.476994
617,41.658028,123.524696
617,41.653019,123.532675


In [1022]:
dfSchoolsClean = pd.concat([dfSchools_5, dfSchools_3], axis=1, sort=False)

In [1023]:
dfSchoolsClean

,lat,lng,name,geometry
0,-9.477105,147.200274,Bava International School & Kindergarten,"{'location': {'lat': -9.477105, 'lng': 147.200..."
0,-9.454446,147.207230,Holy Rosary Primary School,"{'location': {'lat': -9.454445799999998, 'lng'..."
0,-9.463550,147.198858,Coronation Primary School,"{'location': {'lat': -9.463550000000001, 'lng'..."
0,-9.442291,147.189108,Wardstrip Demonstration Primary School,"{'location': {'lat': -9.442291100000002, 'lng'..."
0,-9.474749,147.205147,Bavaroko Primary School,"{'location': {'lat': -9.474749099999999, 'lng'..."
...,...,...,...,...
616,38.951721,121.521942,Dalian Ganjingzi Experimental Primary School,"{'location': {'lat': 38.95172099999999, 'lng':..."
617,41.648409,123.476994,Taoxianzhen Central Elementary School,"{'location': {'lat': 41.648409, 'lng': 123.476..."
617,41.658028,123.524696,宁路小学,"{'location': {'lat': 41.65802800000001, 'lng':..."
617,41.653019,123.532675,Shenyang Conservatory of Music Affiliated Musi...,"{'location': {'lat': 41.653019, 'lng': 123.532..."


In [1024]:
dfSchoolsClean = dfSchoolsClean.dropna()

In [1025]:
dfSchoolsClean

,lat,lng,name,geometry
0,-9.477105,147.200274,Bava International School & Kindergarten,"{'location': {'lat': -9.477105, 'lng': 147.200..."
0,-9.454446,147.207230,Holy Rosary Primary School,"{'location': {'lat': -9.454445799999998, 'lng'..."
0,-9.463550,147.198858,Coronation Primary School,"{'location': {'lat': -9.463550000000001, 'lng'..."
0,-9.442291,147.189108,Wardstrip Demonstration Primary School,"{'location': {'lat': -9.442291100000002, 'lng'..."
0,-9.474749,147.205147,Bavaroko Primary School,"{'location': {'lat': -9.474749099999999, 'lng'..."
...,...,...,...,...
616,38.951721,121.521942,Dalian Ganjingzi Experimental Primary School,"{'location': {'lat': 38.95172099999999, 'lng':..."
617,41.648409,123.476994,Taoxianzhen Central Elementary School,"{'location': {'lat': 41.648409, 'lng': 123.476..."
617,41.658028,123.524696,宁路小学,"{'location': {'lat': 41.65802800000001, 'lng':..."
617,41.653019,123.532675,Shenyang Conservatory of Music Affiliated Musi...,"{'location': {'lat': 41.653019, 'lng': 123.532..."


In [60]:
dfSchoolsClean.columns = ['latitude','longitude','name','geometry']
dfSchoolsClean = dfSchoolsClean.reset_index(drop=True)
dfSchoolsClean

,latitude,longitude,name,geometry
0,-9.477105,147.200274,Bava International School & Kindergarten,POINT (-9.477 147.200)
1,-9.454446,147.207230,Holy Rosary Primary School,POINT (-9.454 147.207)
2,-9.463550,147.198858,Coronation Primary School,POINT (-9.464 147.199)
3,-9.442291,147.189108,Wardstrip Demonstration Primary School,POINT (-9.442 147.189)
4,-9.474749,147.205147,Bavaroko Primary School,POINT (-9.475 147.205)
...,...,...,...,...
4218,38.951721,121.521942,Dalian Ganjingzi Experimental Primary School,POINT (38.952 121.522)
4219,41.648409,123.476994,Taoxianzhen Central Elementary School,POINT (41.648 123.477)
4220,41.658028,123.524696,宁路小学,POINT (41.658 123.525)
4221,41.653019,123.532675,Shenyang Conservatory of Music Affiliated Musi...,POINT (41.653 123.533)


In [34]:
dfSchoolsClean.to_json("output/schoolsCleaned.json", orient="records")

NameError: name 'dfSchoolsClean' is not defined

In [48]:
dfSchoolsClean_1 =pd.read_json('schoolsCleaned.json', orient='records')
dfSchoolsClean_1.columns = ['latitude','longitude','name','geometry']
dfSchoolsClean_1["location"] = dfSchoolsClean_1[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)


In [49]:
dfSchoolsClean_1

,latitude,longitude,name,geometry,location
0,-9.477105,147.200274,Bava International School & Kindergarten,"{'location': {'lat': -9.477105, 'lng': 147.200...","{'type': 'Point', 'coordinates': [147.200274, ..."
1,-9.454446,147.207230,Holy Rosary Primary School,"{'location': {'lat': -9.4544458, 'lng': 147.20...","{'type': 'Point', 'coordinates': [147.2072303,..."
2,-9.463550,147.198858,Coronation Primary School,"{'location': {'lat': -9.46355, 'lng': 147.1988...","{'type': 'Point', 'coordinates': [147.1988583,..."
3,-9.442291,147.189108,Wardstrip Demonstration Primary School,"{'location': {'lat': -9.4422911, 'lng': 147.18...","{'type': 'Point', 'coordinates': [147.1891084,..."
4,-9.474749,147.205147,Bavaroko Primary School,"{'location': {'lat': -9.4747491, 'lng': 147.20...","{'type': 'Point', 'coordinates': [147.205147, ..."
...,...,...,...,...,...
4218,38.951721,121.521942,Dalian Ganjingzi Experimental Primary School,"{'location': {'lat': 38.951721, 'lng': 121.521...","{'type': 'Point', 'coordinates': [121.521942, ..."
4219,41.648409,123.476994,Taoxianzhen Central Elementary School,"{'location': {'lat': 41.648409, 'lng': 123.476...","{'type': 'Point', 'coordinates': [123.4769939,..."
4220,41.658028,123.524696,宁路小学,"{'location': {'lat': 41.658028, 'lng': 123.524...","{'type': 'Point', 'coordinates': [123.524696, ..."
4221,41.653019,123.532675,Shenyang Conservatory of Music Affiliated Musi...,"{'location': {'lat': 41.653019, 'lng': 123.532...","{'type': 'Point', 'coordinates': [123.532675, ..."


In [51]:
dfSchoolsClean_1.to_json("output/schoolsCleaned_1.json", orient="records")

In [40]:
import geopandas as gpd
from cartoframes.auth import set_default_credentials
from cartoframes.viz import Map, Layer,popup_element

set_default_credentials('cartoframes')

In [41]:
dfSchools_points = gpd.GeoDataFrame(dfSchoolsClean_1, geometry=gpd.points_from_xy(dfSchoolsClean_1.longitude, dfSchoolsClean_1.latitude))
airports_points = gpd.GeoDataFrame(airportsCleaned, geometry=gpd.points_from_xy(airportsCleaned.longitude, airportsCleaned.latitude))
starbucks_points = gpd.GeoDataFrame(starbucksCleaned, geometry=gpd.points_from_xy(starbucksCleaned.longitude, starbucksCleaned.latitude))

In [44]:
Map([Layer(airports_points,'color: red'),Layer(starbucks_points, 'color: green' ),Layer(dfSchools_points,'color: yellow',popup_hover=[popup_element('longitude'),popup_element('latitude'),popup_element('name')])])

In [100]:
lst_starbucks = []
for i in airports_points.iterrows():
    lst_starbucks.append(i[1])

In [101]:
a = lst_starbucks[1]
print(a)

name                            Keflavik International Airport
latitude                                                63.985
longitude                                             -22.6056
country                                                     IS
location     {'type': 'Point', 'coordinates': [-22.60560035...
geometry              POINT (-22.605600357056 63.985000610352)
Name: 12289, dtype: object


In [106]:
b = db.starbucks.find( { 'location': { '$geoWithin':{ '$geometry': a.location } } } ).count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


OperationFailure: $within not supported with provided geometry: { $geoWithin: { $geometry: { type: "Point", coordinates: [ -22.605600357056, 63.985000610352 ] } } }

In [107]:
list(b)

[]